# Merging WU Weather Data (30 mins weather interval)
## by : Isaac Harris

In [1]:
import pandas as pd
import glob
import os.path
from pathlib import Path

In [2]:
os.chdir("../Data/raw/WU")
# Create a list of all CSV files
files = glob.glob("*.csv")

# Create an empty list to append the df
filenames = []

for csv in files:
    df = pd.read_csv(csv)
    #df['Date'] = os.path.basename(csv)
    filenames.append(df)

In [3]:
df_final = pd.concat(filenames)

In [4]:
df_final.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,00:00:00,66 °F,55 °F,68 %,ESE,16 mph,0 mph,29.84 in,0.0 in,Cloudy
1,00:30:00,64 °F,57 °F,77 %,E,10 mph,0 mph,29.81 in,0.0 in,Mostly Cloudy
2,01:00:00,64 °F,57 °F,77 %,E,6 mph,0 mph,29.84 in,0.0 in,Mostly Cloudy
3,01:30:00,63 °F,59 °F,88 %,E,7 mph,0 mph,29.81 in,0.0 in,Mostly Cloudy
4,02:00:00,63 °F,59 °F,88 %,ESE,10 mph,0 mph,29.81 in,0.0 in,Mostly Cloudy


In [5]:
df_final['Time'] = pd.to_datetime(df_final['Time'])

In [6]:
sel_col = ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.']

for col in sel_col: 
    df_final[col] = df_final[col].str.extract('(\d+)', expand=False)

In [7]:
df_final.isna().sum()

Time           0
Temperature    0
Dew Point      6
Humidity       0
Wind           0
Wind Speed     0
Wind Gust      0
Pressure       1
Precip.        0
Condition      0
dtype: int64

In [8]:
col_has_na = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Pressure', 'Condition']

for col in col_has_na:
    df_final[col] = df_final[col].fillna(method = 'bfill')

df_final.isna().sum()    

Time           0
Temperature    0
Dew Point      0
Humidity       0
Wind           0
Wind Speed     0
Wind Gust      0
Pressure       0
Precip.        0
Condition      0
dtype: int64

In [9]:
df_final['Temperature'] = round((df_final['Temperature'].astype(int) - 32) / (9/5),2)
df_final['Dew Point'] = round((df_final['Dew Point'].astype(int) - 32) / (9/5),2)

In [9]:
df_final = df_final.rename(columns = { 'Time' : 'datetime', 'Temperature' :'temperature(celcius)', 'Dew Point': 'dew_point(celcius)', 'Humidity' :'humidity(%)',
                                        'Wind Speed': 'wind_speed(mph)', 'Wind Gust': 'wind_gush(mph)', 'Pressure': 'pressure(in)', 'Precip.' : 'precip.(in)'})

In [10]:
path = Path(os.getcwd())
levels_up = 1
to_folder = path.parents[levels_up]
os.chdir(to_folder)
df.to_csv( "edited/WU_merged/wu_weather_merged.csv", index=False, encoding='utf-8-sig')